In [1]:
#Calling neccessary packages 
from math import *
from numpy import *
from pandas import *
from qiskit import *
from qutip import *
import qiskit.quantum_info
import qiskit.visualization
import qiskit.result
from qiskit_aer import AerSimulator
#from qiskit_ibm_runtime import QiskitRuntimeService, Sampler
from qiskit.circuit import SwitchCaseOp
from qiskit.circuit.library import UnitaryGate
from qiskit.visualization import *
from qiskit.result import marginal_counts
from qiskit_ibm_provider import IBMProvider
#from qiskit.providers.ibmq.job import job_monitor
#QiskitRuntimeService.save_account(channel="ibm_quantum", token="4aa3e6491eafc0c9915717ef7dd214e50c3cd61dc74cb27f3edb2941e70e5262c9855be3567ef414eee7d0be34fb189e39a9fce92a59e7dc3c78717a40ec1c69", overwrite=True)
#provider = IBMProvider()

c:\Users\Navid Elyasi\AppData\Local\Programs\Python\Python310\lib\site-packages\qutip\__init__.py:66: UserWarning: The new version of Cython, (>= 3.0.0) is not supported.
  warnings.warn(


In [2]:
#Charging Operator
# Initially we have to define the Hamiltonian 
omega = 1/2
kappa = 1

H = omega * (tensor(sigmax(), qeye(2))) + kappa * (
tensor(sigmam(), sigmap()) + tensor(sigmap(), sigmam()))
# Generating the Unitary gate which is responsible for charging the battey and correlating it with the ancilla 
M_Unitary = (-1j * H).expm()
M_gate = UnitaryGate(M_Unitary)

In [3]:
from qiskit_aer import AerSimulator
import pandas as pd
import numpy as np
backend = AerSimulator()

In [4]:


def get_values_from_csv(file_path, row_number):
    try:
        # Read the csv file into a DataFrame, specifying that the first column may contain complex numbers
        df = pd.read_csv(file_path, header=None)

        # Check if the specified row_number is valid
        if 0 <= row_number < df.shape[0]:
            # Get the values of the specified row
            row_values = df.iloc[row_number, :].tolist()
            return row_values
        else:
            print("Invalid row number. Please enter a valid row number.")
            return None

    except FileNotFoundError:
        print(f"File not found at the path: {file_path}")
        return None
    except Exception as e:
        print(f"An error occurred: {str(e)}")
        return None



In [5]:
#Adressing directory of the csv file
csv_file_path = "output-no.csv"

In [6]:
#Initial Values 
Steps = 10
Energy = []
Passive = []
#Circuit setup

q0 = QuantumRegister(1, name = 'battery')
q1 = QuantumRegister(1, name = 'ancilla')
creg  = ClassicalRegister(1)
qc = QuantumCircuit(q0, q1, creg)
#Creat an empty list to save the measurement results 
result_  = []
j = 0
#Start the collisional model for arbitrary number of steps 
for step in range(0, Steps):
    j = j + 1
    q0 = QuantumRegister(1, name = 'battery')
    q1 = QuantumRegister(1, name = 'ancilla')
    creg  = ClassicalRegister(1)
    qc = QuantumCircuit(q0, q1, creg)
    for i in range(j):
        qc.append(M_gate, [q1, q0])
        qc.barrier()
        qc.reset(q1)
        qc.barrier()
    qc.measure(q0, creg[0])
    qc = transpile(qc, backend = backend)
    job = backend.run(qc, shots = 10000)
    results = job.result()
    counts  = results.get_counts()
    E  = 1 - counts['0']/10000
    Energy.append(E)
    print(E)


    






0.16379999999999995
0.31089999999999995
0.34850000000000003
0.3325
0.32689999999999997
0.31989999999999996
0.32920000000000005
0.3255
0.3328
0.32330000000000003


In [7]:
#Initial Values 
Steps = 10


#Circuit setup
q0 = QuantumRegister(1, name = 'battery')
q1 = QuantumRegister(1, name = 'ancilla')
creg  = ClassicalRegister(1)
qc = QuantumCircuit(q0, q1, creg)

result_  = []
j = 0
#Start the collisional model for arbitrary number of steps 
for step in range(0, Steps):
    j = j + 1
    q0 = QuantumRegister(1, name = 'battery')
    q1 = QuantumRegister(1, name = 'ancilla')
    creg  = ClassicalRegister(1)
    qc = QuantumCircuit(q0, q1, creg)
    for i in range(j):
        qc.append(M_gate, [q1, q0])
        qc.barrier()
        qc.reset(q1)
        qc.barrier()
    Values = get_values_from_csv(csv_file_path, step)
    Unit = Values[0]
    if Unit == 1:
        qc.x(q0)
    qc.measure(q0, creg[0])
    qc = transpile(qc, backend = backend)
    job = backend.run(qc, shots = 10000)
    results = job.result()
    counts = results.get_counts()
    E = counts['1']/10000
    Passive.append(E)
for k in range(len(Energy)):
    Ergotropy = Energy[k] - Passive[k]
    if Ergotropy < 0:
       Ergotropy = 0
    print(Ergotropy)

0
0
0
0
0
0
0
0.006300000000000028
0.009399999999999964
0
